# This notebook stores useful code blocks for ZND Remote Control

## Manual Control
Add buttons on touchscreen to enable manual measuring, memorizing sweeps, clearing them, or exporting them all at once.

In [ ]:
from Instrument import Instrument
from datetime import datetime
import Types
from time import sleep

In [ ]:
# connect to the instrument
resource_string = 'TCPIP::192.168.0.169::hislip0'
instrument = Instrument(resource_string)
instrument.greet()

In [ ]:
def index_sequence(start_index):
    index = start_index
    while True:
        yield index
        index += 1

In [ ]:
def measure():
    instrument.sweep_initiate_all()

In [ ]:
def memorize():
    mem_trace_name = f'MemoryTrace{next(mem_index_gen)}'
    instrument.trace_copy(mem_trace_name, 'MainTrace')
    instrument.trace_assign_to_window(1, next(trc_index_gen), mem_trace_name)

In [ ]:
def export():
    save_path = r'C:\Users\Instrument\Desktop\Data\Manual_{}.dat'.format(str(datetime.now().strftime("%b-%d-%Y_%H-%M-%S")))
    instrument.trace_save_all(1, save_path,
                             formatted=True,
                             save_format=Types.SaveFormat.COMPLEX,
                             dec_separator=Types.DecimalSeparator.POINT,
                             field_separator=Types.FieldSeparator.SEMICOLON)

In [ ]:
def clear():
    global mem_index_gen, trc_index_gen
    instrument.trace_delete_all()
    instrument.trace_create(1, 'MainTrace', 'S21')
    instrument.trace_assign_to_window(1, 1, 'MainTrace')
    instrument.trace_format(1, Types.TraceFormat.LINEAR)
    instrument.sweep_initiate_all()
    instrument.trace_scale_auto(1)
    mem_index_gen = index_sequence(1)
    trc_index_gen = index_sequence(2)

In [ ]:
# set up the instrument
instrument.reset()
instrument.trace_format(1, Types.TraceFormat.LINEAR)
instrument.trace_rename(1, 'MainTrace')
instrument.correction_load(1, 'SMA Ideal.cal')
instrument.trace_set_points(1, 1000)
instrument.sweep_set_mode(1, Types.SweepMode.SINGLE)
instrument.sweep_initiate_all()
instrument.trace_scale_auto(1)
# TODO: set trace points to 1000 *
# TODO: set timeouts
# TODO: set calibration data *
# TODO: set display mode 
# TODO: set trace format *
mem_index_gen = index_sequence(1)
trc_index_gen = index_sequence(2)

In [ ]:
# define buttons for manual control
instrument.button_define(Types.ButtonNumber.ONE, "Measure", measure)
instrument.button_define(Types.ButtonNumber.TWO, "Memorize", memorize)
instrument.button_define(Types.ButtonNumber.THREE, "Export", export)
instrument.button_define(Types.ButtonNumber.FOUR, "Clear", clear)
instrument.button_start_listening()

## Autopilot
Configure instrument and sweep every **sleep_time** seconds, total of **export_length** times, measuring S21 parameter of the antenna and saving it in "Data" folder on Desktop. Output has complex format. Process can be paused, resumed or stopped using buttons on the touchscreen.

In [ ]:
from Instrument import Instrument
from datetime import datetime
import Types
from time import sleep

In [ ]:
# connect to the instrument
resource_string = 'TCPIP::192.168.0.169::hislip0'
instrument = Instrument(resource_string)
instrument.greet()

In [ ]:
def index_sequence(start_index):
    index = start_index
    while True:
        yield index
        index += 1

In [ ]:
paused = False
stopped = False

def pause():
    global paused
    paused = True
    
def resume():
    global paused
    paused = False
    
def stop():
    global stopped
    stopped = True

In [ ]:
# add pause and resume buttons
instrument.button_define(Types.ButtonNumber.ONE, "Pause", pause)
instrument.button_define(Types.ButtonNumber.TWO, "Resume", resume)
instrument.button_define(Types.ButtonNumber.THREE, "Stop", stop)

In [ ]:
# set up the instrument
instrument.reset()
instrument.trace_format(1, Types.TraceFormat.SMITH)
instrument.trace_rename(1, 'MainTrace')
instrument.correction_load(1, 'SMA Ideal.cal')
instrument.trace_set_points(1, 1000)
instrument.sweep_set_mode(1, Types.SweepMode.SINGLE)
instrument.display_set_mode(Types.DisplayMode.OFF) # comment this for debugging
instrument.sweep_initiate_all()
instrument.trace_scale_auto(1)
mem_index_gen = index_sequence(1)
trc_index_gen = index_sequence(2)

In [ ]:
# write autopilot loop
export_length = 50
sleep_time = 500e-3

while True:
    if stopped:
        break
    if paused:
        continue
    
    for _ in range(export_length - 1):
        instrument.sweep_initiate_all()
        mem_trace_name = f'MemoryTrace{next(mem_index_gen)}'
        instrument.trace_copy(mem_trace_name, 'MainTrace')
        # instrument.trace_assign_to_window(1, next(trc_index_gen), mem_trace_name) # uncomment this for debugging
        sleep(sleep_time)
    instrument.sweep_initiate_all()
        
    save_path = r'C:\Users\Instrument\Desktop\Data\Auto_{}.dat'.format(str(datetime.now().strftime("%b-%d-%Y_%H-%M-%S")))
    instrument.trace_save_all(1, save_path,
                             formatted=True,
                             save_format=Types.SaveFormat.COMPLEX,
                             dec_separator=Types.DecimalSeparator.POINT,
                             field_separator=Types.FieldSeparator.SEMICOLON)
    
    instrument.trace_delete_all()
    instrument.trace_create(1, 'MainTrace', 'S21')
    # instrument.trace_assign_to_window(1, 1, 'MainTrace') # uncomment this for debugging
    instrument.trace_format(1, Types.TraceFormat.SMITH)
    instrument.sweep_initiate_all()
    instrument.trace_scale_auto(1)
    mem_index_gen = index_sequence(1)
    trc_index_gen = index_sequence(2)

## Steppers

In [ ]:
from Steppers import Steppers
from time import sleep

In [ ]:
steppers = Steppers()

In [ ]:
steppers.release()
steppers.attach()
steppers.set_home()
steppers.set_ignore_bounds(True)
steppers.move(10, 0)
steppers.set_max()

In [ ]:
steppers.move(1, 1)
sleep(1)
steppers.move(-1, -1)
sleep(1)
steppers.set_position(5, 5)
sleep(1)
steppers.home()
sleep(1)

In [ ]:
steppers.get_position()

In [ ]:
steppers.move(10, 10)
steppers.set_home()
steppers.set_position(0, 0)
steppers.set_home()